In [200]:
import re

import jsonlines
import pandas as pd
import numpy as np

In [201]:
# # TODO: load metadata and return aggregate / summary statistics
# # TODO: write method to display cover image w/ cover metadata and add annotations to image
# # TODO: consider ideate/innotater for annotating directly in Jupyter notebooks

logged_metadata = []

with jsonlines.open('../metadata/covers.jsonl', mode='r') as reader:
    for item in reader:
        logged_metadata.append(item)
        
df = pd.DataFrame(logged_metadata)

In [202]:
def get_issue_number_from_title(title):
    issue = re.search(r"([#?])(\d+)\b", title.replace(',', ''))
    if issue is None:
        return np.nan
    else:
        return np.int(issue.group().replace('#', ''))

df['issue_number'] = df['title'].apply(get_issue_number_from_title)

df.drop(['issue_number', 'covers'], axis=1).describe().T

,count,unique,top,freq
format_binding,9229,25,Saddle-stitched,4628
format_color,9229,9,Color,5895
format_dimensions,9229,30,Standard Modern Age U.S.,1953
format_paper_stock,9229,28,,3244
format_publishing_format,9229,27,Was Ongoing Series,4301
indexer_notes,9229,6355,This issue has variants. |,1277
indicia_frequency,9229,240,monthly,5237
issue_brand,9229,83,DC [bullet],1719
issue_indicia_publisher,9229,41,DC Comics,2426
issue_pages,9229,56,36,6326


In [203]:
df_covers = pd.concat(df['covers'].apply(lambda x: pd.DataFrame(x).T).tolist(), axis=0).reset_index(drop=True)

df_covers.tail()

,cover_characters,cover_colors,cover_editing,cover_first line of dialogue or text,cover_genre,cover_inks,cover_job number,cover_keywords,cover_letters,cover_pencils,cover_script,cover_synopsis,image_url,save_to
12424,"Namor, the Sub-Mariner; Namorita; Dorma",?,NaN,NaN,superhero,John Byrne (signed),NaN,NaN,?,John Byrne (signed),?,NaN,https://files1.comics.org//img/gcd/covers_by_i...,"./covers/Namor, the Sub-Mariner: Namor, the Su..."
12425,"Namor, the Sub-Mariner; Namorita",?,NaN,NaN,superhero,John Byrne (signed),NaN,NaN,NaN,John Byrne (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,"./covers/Namor, the Sub-Mariner: Namor, the Su..."
12426,"Namor, the Sub-Mariner; Namorita; Misty Knight...",?,NaN,NaN,superhero,John Byrne (signed),NaN,NaN,?,John Byrne (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,"./covers/Namor, the Sub-Mariner: Namor, the Su..."
12427,"Namor, the Sub-Mariner; Iron Fist (?); H'ylthri",?,NaN,NaN,superhero,John Byrne (signed),NaN,NaN,?,John Byrne (signed),?,NaN,https://files1.comics.org//img/gcd/covers_by_i...,"./covers/Namor, the Sub-Mariner: Namor, the Su..."
12428,"Namor, the Sub-Mariner; Misty Knight; Iron Fis...",?,NaN,NaN,superhero,John Byrne (signed),NaN,NaN,?,John Byrne (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,"./covers/Namor, the Sub-Mariner: Namor, the Su..."


In [204]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [241]:
def replace_semicolon_in_brackets(s: str) -> str:
    """
    Return the substring of the first instance of bracketed text.
    """
    regex_brackets = re.search(r"(?<=\[).*?(?=\])", s)
    if regex_brackets is None:
        return None
    else:
        return regex_brackets.group()

In [243]:
vals = list(df_covers[df_covers["cover_characters"].fillna("").apply(lambda x: "Kal-El]" in x)]["cover_characters"].values)

[replace_semicolon_in_brackets(x) for x in  vals]
vals

['Justice League [Aquaman [Arthur Curry]; Batman [Bruce Wayne]; Cyborg [Victor Stone]; Flash [Barry Allen]; Superman [Clark Kent; Kal-El]; Wonder Woman [Diana]]',
 'Justice League [Aquaman [Arthur Curry]; Batman [Bruce Wayne]; Cyborg [Victor Stone]; Flash [Barry Allen]; Superman [Clark Kent; Kal-El]; Wonder Woman [Diana]]',
 'Justice League [Aquaman [Arthur Curry]; Batman [Bruce Wayne]; Cyborg [Victor Stone]; Flash [Barry Allen]; Superman [Clark Kent; Kal-El]; Wonder Woman [Diana]]',
 'Superman [Clark Kent; Kal-El]',
 'Superman [Clark Kent; Kal-El]',
 'Batman [Bruce Wayne]; Superman [Clark Kent; Kal-El]',
 'Batman [Bruce Wayne]; Superman [Clark Kent; Kal-El]',
 'Batman [Bruce Wayne]; Superman [Clark Kent; Kal-El]',
 'Batman [Bruce Wayne]; Superman [Clark Kent; Kal-El]',
 'Batman [Bruce Wayne]; Superman [Clark Kent; Kal-El]',
 'Batman [Bruce Wayne]; Batman [Bruce Wayne] (Earth-2); Superman [Clark Kent; Kal-El]; Superman [Clark Kent] (Earth-2)',
 'Batman [Bruce Wayne]; Superman [Clark Ke

In [205]:
# get list of unique characters across all  covers
def get_value_counts(df, column):
    """
    """
    return pd.Series("; ".join(df[column].\
                               dropna()).\
                     replace(" (signed)", "").\
                     replace(" (painted)", "").\
                     split('; ')).value_counts()

get_value_counts(df_covers, 'cover_characters')[:20]

Batman [Bruce Wayne]              1011
Spider-Man [Peter Parker]          856
Superman                           609
Batman                             565
Conan                              515
Human Torch [Johnny Storm]         409
Iron Man [Tony Stark]              345
Robin [Dick Grayson]               253
Wolverine                          240
Captain America [Steve Rogers]     240
Kal-El]                            233
Superman [Clark Kent               222
Fantastic Four [Mr. Fantastic      212
Superman [Clark Kent]              205
Spider-Man                         202
Cyclops                            193
Beast                              181
Cable                              180
Aquaman                            169
Joker                              168
dtype: int64

In [206]:
get_value_counts(df_covers, 'cover_pencils')[:20]

Jack Kirby          330
John Buscema        286
Gil Kane            258
Alex Ross           246
John Byrne          224
John Romita         211
Curt Swan           178
George Pérez        146
?                   136
Paul Ryan           134
Rich Buckler        123
Ryan Ottley         123
Greg Capullo        119
Brian Bolland       110
Nick Cardy          106
Salvador Larroca     97
Andy Kubert          96
Humberto Ramos       94
Jim Aparo            94
Neal Adams           93
dtype: int64

In [207]:
get_value_counts(df,  "series_name")[:20]

Action Comics             946
Batman                    865
The Amazing Spider-Man    733
Fantastic Four            577
The Avengers              409
Conan the Barbarian       293
Aquaman                   267
X-Factor                  251
Avengers                  242
X-Men                     226
Batgirl                   172
The X-Men                 165
X-Force                   163
Alpha Flight              137
Cable                     129
Harley Quinn              119
Animal Man                119
Exiles                    115
Amazing Spider-Man        105
Conan Saga                 99
dtype: int64

In [208]:
get_value_counts(df_covers,  "cover_genre")[:20]

superhero                        10656
sword and sorcery                  640
fantasy-supernatural               255
science fiction                    170
humor                              125
horror-suspense                    102
satire-parody                       78
adventure                           74
crime                               72
jungle                              40
anthropomorphic-funny animals       35
western-frontier                    32
historical                          29
war                                 19
drama                                7
spy                                  6
detective-mystery                    5
children                             4
aviation                             1
sports                               1
dtype: int64

In [188]:
def create_training_dirs():
    """
    Given some args, create two compressed files: images.tar.gz & annos.tar.gz
    """
    
    

In [ ]:
#TODO: make this work (e.g. pull in series to df_covers... probably just all issue metadata honestly...)
def concat_values_by_series(df, concat_column, series_name):    

    list_of_synopsis = df[
        (df['series_name'] == series_name) & df['issue_number'] != 0.0].sort_values('issue_number')[concat_column]

    return " | ".join(list_of_synopsis.values)

print(concat_values_by_series(df, concat_column='synopsis', series_name="Detective Comics")[:8000], '...')